![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 09 - Evaluación del rendimiento de un modelo

Cuando queremos evaluar un modelo, lo primero que se nos viene a la cabeza es entrenar el modelo utilizando unos datos y luego utilizar una métrica (accuracy por ejemplo) para medir el rendimiento de ese modelo sobre los mismos datos que se han utilizado para entrenar. Sin embargo, esto es un error puesto que estaremos obteniendo resultados que sobreestiman el verdadero rendimiento del modelo.

**Una correcta evaluación de un modelo es aquella que utiliza datos distintos a los utilizado durante el entrenamiento.**

En esta práctica vamos a ver cómo podemos hacer evaluaciones de modelos de manera correcta.

## 9.1 Reescritura (Resubstitution)

Si utilizamos todos los datos de que disponemos para entrenar el modelo y, posteriormente, utilizamos esos mismos datos para evaluar el modelo estaremos haciendo lo que se conoce como reescritura o resubstitution.

Veamos con un ejemplo cuál es el problema de esta técnica de evaluación. Para ello vamos a cargar el conjunto 'ionosphere.data' transformando la clase a valores numéricos:

In [1]:
# se importan las librerías
import pandas as pd
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import time

# se llama a la función read_csv
# no tiene missing y las columnas están separadas por ','
# tampoco cabecera, así que hay que dar nombre a las columnas (como en el names no vienen indicados creamos nombres)
cabecera = ['atr'+str(x) for x in range(1,35)] # for en línea para crear elementos de una lista
cabecera.append('clase') # la clase está en la última posición

df = pd.read_csv('ionosphere.data', names=cabecera)
filas, columnas = df.shape

# la clase está en la última columna 
# separamos los atributos y los almacenamos en X
X = df.drop(['clase'], axis=1)
display(X)

# transformamos la clase
class_enc = preprocessing.LabelEncoder()
df['clase'] = class_enc.fit_transform(df['clase'])

# separamos la clase y la almacenamos en Y
y = df['clase']
display(y)

,atr1,atr2,atr3,atr4,atr5,atr6,atr7,atr8,atr9,atr10,...,atr25,atr26,atr27,atr28,atr29,atr30,atr31,atr32,atr33,atr34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243


0      1
1      0
2      1
3      0
4      1
      ..
346    1
347    1
348    1
349    1
350    1
Name: clase, Length: 351, dtype: int32

Ahora vamos a ver la accuracy que obtiene el sistema que predice siempre la clase mayoritaria:

In [2]:
# creamos el sistema DummyClassifier strategy="most_frequent"
cl_my_sis = DummyClassifier(strategy="most_frequent")
cl_my_sis.fit(X, y)
y_pred = cl_my_sis.predict(X)
print("\n### Resultados para DummyClassifier con strategy='most_frequent' ###")
print("Accuracy :", accuracy_score(y, y_pred))


### Resultados para DummyClassifier con strategy='most_frequent' ###
Accuracy : 0.6410256410256411


Y la accuracy obtenida por un `KNN` con `n_neighbors=1`:

In [3]:
# creamos una instancia del KNN
knn_sis = KNeighborsClassifier(n_neighbors=1)
knn_sis.fit(X, y)
y_pred = knn_sis.predict(X)
print("\n### Resultados para KNeighborsClassifier con n_neighbors=1 ###")
print("Accuracy :", accuracy_score(y, y_pred))


### Resultados para KNeighborsClassifier con n_neighbors=1 ###
Accuracy : 1.0


Vemos que este último sistema tiene un porcentaje de acierto del 100%.

¿Quiere esto decir que hemos entrenado un modelo que es perfecto a la hora de predecir? NO, lo que quiere decir es que la reescritura no es un buen modelo para medir la calidad de nuestro modelo en el futuro.

Si queremos tener una estimación de la calidad de nuestro modelo cuando esté *en producción*, entonces debemos comprobar su rendimiento con casos que no haya visto durante el entrenamiento. Hay varias formas de simular esta situación:
1. Hold-out (entrenamiento-test, validation)
2. Validación cruzada (cross validation)
3. Leave-one-out

## 9.2 Hold-out (entrenamiento-test)

La manera más sencilla de lograr lo que pretendemos es simplemente partir el conjunto de datos (data set) en dos trozos que tradicionalmente se llaman *training set* y *test set* (a veces también *validation set*). 

![Hold-out](fig_holdout.png) 

A esta técnica se la conoce como **Hold-out** (a veces también entrenamiento-test) y normalmente se suelen reservar más ejemplos para el entrenamiento que para el test. Una división típica puede ser 75%-25% aunque dependerá de las particularidades de cada conjunto.

Veamos cómo implementar un hold-out utilizando la función `train_test_split()`:


In [4]:
# se separan train y test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# reentrenamos el baseline
cl_my_sis.fit(X_train, y_train)     # se entrena sobre la partición de train
y_test_pred = cl_my_sis.predict(X_test)  # se predice sobre la partición de test
print("Accuracy clase mayoritaria: %.4f" % accuracy_score(y_test, y_test_pred))

# reentrenamos el vecino más próximo
knn_sis.fit(X_train, y_train)       # se entrena sobre la partición de train
y_test_pred = knn_sis.predict(X_test)    # se predice sobre la partición de test
print("Accuracy KNN con 1 vecino : %.4f" % accuracy_score(y_test, y_test_pred))

Accuracy clase mayoritaria: 0.6023
Accuracy KNN con 1 vecino : 0.8182


Ahora el KNN ya no tiene un acierto del 100% sino que baja a un 85% aproximadamente. Como se está haciendo el test sobre ejemplos que no ha visto en el entrenamiento el modelo debe enfrentarse a casos desconocidos y su resultado ya no es tan bueno, pero ahora sí que estamos generando una situación más parecida a la que tendría el sistema en producción.

Hay varios detalles interesantes en el código anterior:
1. `train_test_split()` retorna 2 elementos por cada elemento que le damos a partir. Como en este caso le damos para dividir `X` e `y` nos devolverá 4 elementos.
2. Podemos indicarle cómo queremos hacer la división utilizando los parámetros `test_size` o `train_size`. 
3. El entrenamiento **debe** hacerse utilizando el train set
4. La predicción **debe** hacerse utilizando el test set

En https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html se pueden ver todos los parámetros que tiene la función.

### 9.2.1 Barajado y estratificado

Si ejecutas varias veces el código anterior verás que cada vez obtendrás resultados diferentes. Esto es así porque antes de hacer la partición, la función `train_test_split()` realiza un barajado de los ejemplos basándose en un generador de números pseudoaleatorios. Como nos interesa que los resultados puedan reproducirse, normalmente incilializaremos el estado del generador de números pseudoaleatorios utilizando el parámetro `random_state`. 

Además, como las particiones se realizan de manera aleatoria, podría darse el caso de que todos los ejemplos de una clase quedasen en el train set y los de la otra clase en el test set y eso dificultaría el aprendizaje del modelo. Para evitar este tipo de situaciones, se suele realizar una partición **estratificada** que tratará de mantener en los conjuntos de entrenamiento y test la misma proporción de ejemplos de cada clase que se tenía en el conjunto de datos original (completo). Para lograr esto utilizaremos el parámetro `stratify` al que le asignaremos el vector que utilizará de guía para realizar la estratificación.

Esta sería una mejor implementación y aunque la ejecutemos varias veces, siempre obtendremos el mismo resultado:

In [5]:
# se separan train y test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234, stratify=y)

# reentrenamos el baseline
cl_my_sis.fit(X_train, y_train)     # se entrena sobre la partición de train
y_test_pred = cl_my_sis.predict(X_test)  # se predice sobre la partición de test
print("Accuracy clase mayoritaria: %.4f" % accuracy_score(y_test, y_test_pred))

# reentrenamos el vecino más próximo
knn_sis.fit(X_train, y_train)       # se entrena sobre la partición de train
y_test_pred = knn_sis.predict(X_test)    # se predice sobre la partición de test
print("Accuracy KNN con 1 vecino : %.4f" % accuracy_score(y_test, y_test_pred))

Accuracy clase mayoritaria: 0.6364
Accuracy KNN con 1 vecino : 0.8864


### 9.2.2 Problemas del Hold-out

El primer problema es que el resultado será muy dependiente de los pocos ejemplos que hemos separado como conjunto de test. ¿Qué pasaría si por azar los ejemplos más fáciles o difíciles de clasificar quedasen todos en el conjunto de test?, pues que el rendimiento obtenido no reflejaría el verdadero rendimiento que el modelo tendría en producción. Esto se podría solucionar dejando más ejemplos para el conjunto de test, pero eso agravaría el segundo problema.

El segundo problema es que no estamos utilizando todos los ejemplos disponibles para entrenar bien al modelo. Estamos reservando unos ejemplos para la evaluación y al no utilizarlos estamos perdiendo conocimiento. Este problema se podría solucionar utilizando más ejemplos para el entrenamiento, pero eso agravaría el primer problema.

Como vemos, esta técnica presenta dos problemas que no puede solucionar a la vez.

## 9.3 Cross validation (Validación cruzada)

Para solucionar los problemas que presenta el Hold-out existe otra técnica de evaluación del rendimiento de un modelo que se conoce como **cross validation** o validación cruzada.

Esta técnica consiste en dividir el conjunto de datos en K *folds* y realizar K hold-out dejando en cada iteración un fold diferente como conjunto de test. Esta manera de evaluar un modelo se llama **K-Fold Cross Validation** (KFCV):

![Cross validation](fig_cv.png) 

Como vemos en la figura, tendremos K resultados que se combinarán utilizando la media para obtener la estimación de rendimiento del sistema.

Utilizando una KFCV se soluciona el primero de los problemas que observamos en el hold-out y se suaviza el segundo: 
- Todos los ejemplos forman parte del conjunto de test en alguna iteración. Así ya no tendremos el problema que veíamos en el hold-out.
- La mayoría de los ejemplos se utilizan en el entrenamiento. Pero no todos, los modelos que se entrenan en la KFCV siguen sin utilizar todos los ejemplos para entrenar los modelos. Cuanto mayor sea el valor de la K, más se suavizará este problema.

Valores típicos para el número de folds son 5 y 10.

Vamos a ver cómo podemos implementarlo utilizando la función `cross_val_score()`:

In [6]:
# se crea un generador de folds estratificados partiendo el conjunto en 5 trozos
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# se realiza la validación cruzada para el baseline
scores_cl_may = cross_val_score(cl_my_sis, X, y, cv=folds, scoring='accuracy')
print("Clase mayoritaria:", scores_cl_may)

# se toman tiempos de ejecución para la validación cruzada del KNN
ini = time.time()
scores_knn = cross_val_score(knn_sis, X, y, cv=folds, scoring='accuracy')
fin = time.time()
print("KNN con 1 vecino :", scores_knn)
print("\nTiempo del KNN: %.2f segundos" % (fin-ini))

print("\nClase mayoritaria (mean+-std): %0.4f +- %0.4f" % (scores_cl_may.mean(), scores_cl_may.std()))
print("KNN con 1 vecino  (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))

Clase mayoritaria: [0.63380282 0.64285714 0.64285714 0.64285714 0.64285714]
KNN con 1 vecino : [0.87323944 0.82857143 0.85714286 0.87142857 0.88571429]

Tiempo del KNN: 0.05 segundos

Clase mayoritaria (mean+-std): 0.6410 +- 0.0036
KNN con 1 vecino  (mean+-std): 0.8632 +- 0.0196


Como vemos, a `cross_val_score()` tenemos que pasarle el sistema, la matriz de atributos, la clase, los folds que queremos utilizar y la métrica para la que queremos obtener el resultado. En https://scikit-learn.org/stable/modules/model_evaluation.html podemos ver el nombre de las métricas y las funciones asociadas.

En el parámetro `cv` de la función le estamos pasando un `StratifiedKFold`, que va a permitir generar folds partiendo el conjunto de datos en `n_splits` trozos, barajando previamente los ejemplos. Además, fijamos la semilla de números aleatorios para que el experimento sea reproducible.
Podríamos haber indicado directamente `cv=5` en la función `cross_val_score()` y también generaría los folds utilizando `StratifiedKFold` cuando estamos con conjuntos de datos de clasificación, sin embargo, **no barajaría los ejemplos**. Barajar los ejemplos es importante para evitar sesgos, con lo que recomendamos barajar los ejemplos por nuestra cuenta antes de llamar a `cross_val_score()` o utilizar `StratifiedKFold` con `shuffle=True`:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_score#sklearn.model_selection.cross_val_score
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold


Si os fijáis hemos tomado tiempos para saber cuánto tarda en ejecutarse puesto que vamos a comparar tiempos más adelante.

`cross_val_score()` es la manera más sencilla de hacer una KFCV en `scikit-learn`, pero hay otras formas más versátiles. Por ejemplo, utilizando `cross_validate()` podríamos obtener el resultado de varias métricas simultáneamente y además obtendríamos un desglose de tiempos (entrenamiento y predicción): https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate Pero en esta asignatura vamos a centrarnos en el uso básico de `cross_val_score()`.

Vamos a ver ahora qué resultados se obtienen con K=10:

In [7]:
# se crea un generador de folds estratificados partiendo el conjunto en 10 trozos
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# se realiza la validación cruzada para el baseline
scores_cl_may = cross_val_score(cl_my_sis, X, y, cv=10, scoring='accuracy')
print("Clase mayoritaria:", scores_cl_may)

# se toman tiempos de ejecución para la validación cruzada del KNN
ini = time.time()
scores_knn = cross_val_score(knn_sis, X, y, cv=folds, scoring='accuracy')
fin = time.time()
print("KNN con 1 vecino :", scores_knn)
print("\nTiempo del KNN: %.2f segundos" % (fin-ini))

print("\nClase mayoritaria (mean+-std): %0.4f +- %0.4f" % (scores_cl_may.mean(), scores_cl_may.std()))
print("KNN con 1 vecino  (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))

Clase mayoritaria: [0.63888889 0.65714286 0.65714286 0.65714286 0.65714286 0.62857143
 0.62857143 0.62857143 0.62857143 0.62857143]
KNN con 1 vecino : [0.83333333 0.94285714 0.91428571 0.85714286 0.88571429 0.82857143
 0.8        0.91428571 0.88571429 0.91428571]

Tiempo del KNN: 0.08 segundos

Clase mayoritaria (mean+-std): 0.6410 +- 0.0135
KNN con 1 vecino  (mean+-std): 0.8776 +- 0.0439


Vemos que los resultados son bastante similares y que el tiempo de ejecución se ha incrementado.

## 9.4 Leave-one-out

Y así es como llegamos a la mejor estimación que podemos conseguir, que es utilizando la técnica que se conoce como **leave-one-out** y que no es más que una K-Fold Cross Validation en la que la K es igual al número de ejemplos.

De esta manera se realizarán tantos entrenamientos como número de ejemplos tenga el conjunto de datos, dejando en cada iteración un solo ejemplo para el conjunto de test. Esto es lo más parecido que podemos tener a un sistema en producción puesto que se utilizan todos los ejemplos menos uno para entrenar el modelo. Además, todos los ejemplos estarán una vez en el conjunto de test, con lo que la media del rendimiento que tengamos al final incluirá a todos los ejemplos, ya sean fáciles o difíciles. 

**Problema:** Se realizarán tantos entrenamientos como número de ejemplos tengamos en el conjunto y esto hace que la mayoría de las veces el leave-one-out sea impracticable.

In [8]:
# se realiza el leave-one-out para el baseline
scores_cl_may = cross_val_score(cl_my_sis, X, y, cv=LeaveOneOut(), scoring='accuracy')
print("Clase mayoritaria:", scores_cl_may)

# se toman tiempos de ejecución para el leave-one-out del KNN
ini = time.time()
scores_knn = cross_val_score(knn_sis, X, y, cv=LeaveOneOut(), scoring='accuracy')
fin = time.time()
print("KNN con 1 vecino :", scores_knn)
print("\nTiempo del KNN: %.2f segundos" % (fin-ini))

print("\nClase mayoritaria (mean+-std): %0.4f +- %0.4f" % (scores_cl_may.mean(), scores_cl_may.std()))
print("KNN con 1 vecino  (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))

Clase mayoritaria: [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

Podemos resaltar varias cosas en el resultado obtenido:
1. El leave-one-out necesita mucho más tiempo de ejecución.
2. El resultado mejora puesto que estamos utilizando más ejemplos para entrenar los modelos. Esto no siempre será así, pero si nuestro sistema es bueno y le damos más ejemplos para entrenar, debería sacar provecho de esos datos y generar un modelo mejor.
3. El resultado de cada fold es 1 o 0 puesto que al haber un solo ejemplo en el test o se acierta o se falla (en los problemas de clasificación).

En cuanto a la implementación, vemos que para ejecutar un leave-one-out únicamente tenemos que utilizar la función `cross_val_score()` indicando en el parámetro `cv` lo siguiente: `cv=LeaveOneOut()`. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html 

### 9.4.1 Paralelizar ejecuciones

Muchos sistemas y muchas funciones de `scikit-learn` nos dan la posibilidad de paralelizar y sacar el máximo provecho a nuestras CPUs. Lo hacen a través del parámetro `n_jobs` en el que podemos indicarle el número de CPUs que queremos utilizar. Si queremos utilizar todas las CPUs entonces debemos poner `n_jobs=-1`:

In [13]:
# se toman tiempos de ejecución para el leave-one-out del KNN
ini = time.time()
scores_knn = cross_val_score(knn_sis, X, y, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1)
fin = time.time()
print("KNN con 1 vecino :", scores_knn)
print("\nTiempo del KNN: %.2f segundos" % (fin-ini))

print("KNN con 1 vecino  (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))

KNN con 1 vecino : [1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

La primera vez que lo ejecutes igual tarda un poco más por cuestiones de inicializaciones, pero si vuelves a repetir la ejecución verás que el tiempo se reduce considerablemente.

## 9.5 Cómo preprocesar los datos cuando tenemos un conjunto de test (MUY IMPORTANTE)

En los métodos de evaluación que hemos comentado, los modelos siempre se entrenan con unos ejemplos determinados (train set) y se evalúa su rendimiento con otros ejemplos que no se ven durante el entrenamiento (test set). 

Actuamos de esta manera porque queremos simular una situación real en la que se entrena el modelo con todos los ejemplos disponibles hasta un instante determinado y a partir de ese momento comienza a realizar predicciones para los ejemplos que comienzan a llegar a partir de ese instante.

Por tanto, si nuestro sistema requiere realizar algún preprocesado de los datos, los parámetros del preprocesado se deben ajustar sobre los ejemplos del conjunto de entrenamiento únicamente. Si incluyésemos los del conjunto de test estaríamos utilizando datos que supuestamente no podríamos estar viendo.

### 9.5.1 Preprocesado y hold-out
Veamos cómo se haría un hold-out con preprocesado y para ello vamos a utilizar un conjunto que ya habíamos visto en la práctica anterior que tenía los atributos con órdenes de magnitud diferentes:

In [16]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='datos')
filas, columnas = df.shape

# separamos las primeras columnas y las almacenamos en X
X = df.iloc[:,0:(columnas-1)]

# separamos la clase
y = df.iloc[:,(columnas-1)]

# realizamos la partición
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, test_size=0.25)

# creamos el StandardScaler
std_sca = preprocessing.StandardScaler()

# se entrena el StandardScaler SOLO con el conjunto de entrenamiento
std_sca.fit(X_train)

# se transforma el conjunto de entrenamiento y el de test
X_train_std = std_sca.transform(X_train)
X_test_std = std_sca.transform(X_test)

# calculamos la accuracy del baseline para tener una referencia
cl_my_sis.fit(X_train_std, y_train)
y_pred = cl_my_sis.predict(X_test_std)
print("Accuracy clase mayoritaria:", accuracy_score(y_test, y_pred))

# calculamos la accuracy del KNN
knn_sis.fit(X_train_std, y_train)    # se utilizan los atributos ya escalados
y_pred = knn_sis.predict(X_test_std) # se utilizan los atributos ya escalados
print("Accuracy KNN con 1 vecino :", accuracy_score(y_test, y_pred))

Accuracy clase mayoritaria: 0.36
Accuracy KNN con 1 vecino : 0.64


c:\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Como vemos, el `StandardScaler` utiliza únicamente los ejemplos del conjunto de entrenamiento para calcular la media y la desviación que necesita para escalar los datos. Una vez calculada la media y la desviación debemos utilizar `transform` para escalar el conjunto de entrenamiento y el de test.

Esos conjuntos transformados serán los que utilizaremos para entrenar el modelo y realizar las predicciones. Aquí debemos ser cuidadosos, ya que si entrenamos con `X_train_std` y predecimos para los casos contenidos en `X_test` (nos hemos olvidado de escalarlos) los resultados que obtendremos serán muy malos. Puedes probar.

### 9.5.2 Preprocesado y validación cruzada o leave-one-out

Si en lugar de un hold-out queremos realizar una validación cruzada o un leave-one-out, entonces debemos crear un `Pipeline` y utilizarlo como un sistema más. No hay que hacer nada especial.

Vamos a ver cómo se hace una validación cruzada sin estandarizar y estandarizando para que veamos que la única diferencia es que hay que crear el `Pipeline`:

(en este ejemplo ponemos directamente el número del folds que queremos utilizar, pero ya comentamos más arriba que esto puede resultar peligroso puesto que, aunque se llama por defecto a `StratifiedKFold`, no se barajarían los ejemplos antes de separarlos)

In [ ]:
# se realiza la validación cruzada para el baseline
scores_cl_may = cross_val_score(cl_my_sis, X, y, cv=10, scoring='accuracy')

# se toman tiempos de ejecución para la validación cruzada del KNN (sin estandarizar)
ini = time.time()
scores_knn = cross_val_score(knn_sis, X, y, cv=10, scoring='accuracy')
fin = time.time()
print("Tiempo del KNN: %.2f segundos" % (fin-ini))

# se crea un Pipeline que estandariza y usa un KNN
std_knn = Pipeline([('std', std_sca), ('knn', knn_sis)])

# se toman tiempos de ejecución para la validación cruzada del KNN (estandarizando)
ini = time.time()
scores_std_knn = cross_val_score(std_knn, X, y, cv=10, scoring='accuracy')
fin = time.time()
print("Tiempo del STD_KNN: %.2f segundos" % (fin-ini))

print("\nClase mayoritaria     (mean+-std): %0.4f +- %0.4f" % (scores_cl_may.mean(), scores_cl_may.std()))
print("KNN con 1 vecino      (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))
print("STD_KNN con 1 vecino  (mean+-std): %0.4f +- %0.4f" % (scores_std_knn.mean(), scores_std_knn.std()))

En este caso ya se encarga `cross_val_score()` de entrenar los modelos con los conjuntos adecuados y de realizar las predicciones sobre los conjuntos ya estandarizados.

## Ejercicios

1. Carga el fichero **heart_failure_clinical_records_dataset.csv** (es un archivo de texto). 
2. Crea el baseline de la clase mayoritaria y un KNN con 3 vecinos
3. Haz un hold-out con una partición 75%-25% (ten en cuenta que los atributos tienen escalas diferentes).
4. Haz una validación cruzada de 10 folds.
5. Haz un leave-one-out.
6. Haz una gráfica en la que se vea la evolución (en función del número de vecinos [1..10]) de la accuracy obtenida en una validación cruzada de 10 folds estandarizando y sin estandarizar.

Estos ejercicios no es necesario entregarlos.